In [4]:
# import

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

In [16]:
website = 'website URL '
accont = 'your accont'
password = 'your password'
destDir = 'your destnition to download files'

with open('.credential/website') as wb:
    website = wb.readline()

with open('.credential/accont') as ac:
    accont = ac.readline()

with open('.credential/password') as pw:
    password = pw.readline()

with open('.credential/destDir') as dd:
    destDir = dd.readline()

In [20]:
# 啟動瀏覽器工具的選項
my_options = webdriver.EdgeOptions()
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

driver = webdriver.Edge(options=my_options)

In [26]:
# functions for driver

# visit a website
def visit(URL):
    driver.get(URL)
    print('Going to', website)

# wait till some element shows up
def wait_Element(CSS_target):
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, CSS_target)
            )
        )

    except TimeoutException:
        print('ERROR: Time Out!')



In [27]:
# main

if __name__ == '__main__':
    visit(website)
    wait_Element('button.btn.btn-primary.btn-lg.btn-block') #登入按鈕

Going to https://ncueeclass.ncu.edu.tw/dashboard/historyCourse
fffuck
